In [9]:
import csv
from pathlib import Path

from sklearn.metrics import cohen_kappa_score

FIELD_NAMES = ['Pattern', 'Sub Pattern']
COLUMN_MAP = {
    'Pattern': {'Huiyun': 'Huiyun_Pattern', 'Ricardo': 'Ricardo_Pattern'},
    'Sub Pattern': {'Huiyun': 'Huiyun_Sub_Pattern', 'Ricardo': 'Ricardo_Sub_Pattern'},
}
DATASETS = [
    {
        'label': 'final human labels',
        'file': Path('./Final_Human_PRs.csv'),
    },
    {
        'label': 'final agent labels',
        'file': Path('./Final_Agent_PRs.csv'),
    },
]


def load_table(path):
    with path.open(newline='', encoding='utf-8') as f:
        return {row['PR ID']: row for row in csv.DictReader(f)}


def collect_labels(table, field):
    labels_h, labels_r = [], []
    columns = COLUMN_MAP[field]
    for row in table.values():
        a = row.get(columns['Huiyun'], '').strip()
        b = row.get(columns['Ricardo'], '').strip()
        if a and b:
            labels_h.append(a)
            labels_r.append(b)
    return labels_h, labels_r


def compute_metrics(table, field):
    labels_h, labels_r = collect_labels(table, field)
    total = len(labels_h)
    if not total:
        return 0, 0.0, float('nan')
    matches = sum(1 for a, b in zip(labels_h, labels_r) if a == b)
    match_rate = matches / total
    kappa = cohen_kappa_score(labels_h, labels_r)
    return total, match_rate, kappa


for dataset in DATASETS:
    print(f"Cohen's Kappa Agreement (Huiyun vs Ricardo {dataset['label']})")
    print("Identifier columns used: ['PR ID']")
    table = load_table(dataset['file'])
    for field in FIELD_NAMES:
        total, match_rate, kappa = compute_metrics(table, field)
        print(f"Rows compared: {total}")
        field_name = field.lower().replace(' ', '_')
        print(f"  {field_name}: kappa={kappa:.4f}, match_rate={match_rate * 100:.2f}%")


Cohen's Kappa Agreement (Huiyun vs Ricardo final human labels)
Identifier columns used: ['PR ID']
Rows compared: 40
  pattern: kappa=0.6581, match_rate=70.00%
Rows compared: 26
  sub_pattern: kappa=0.5759, match_rate=61.54%
Cohen's Kappa Agreement (Huiyun vs Ricardo final agent labels)
Identifier columns used: ['PR ID']
Rows compared: 155
  pattern: kappa=0.6010, match_rate=65.81%
Rows compared: 101
  sub_pattern: kappa=0.5988, match_rate=62.38%


In [11]:
import csv
from pathlib import Path

csv.field_size_limit(1_000_000_000)

ROUND3_ROOT = Path('../llm_data/new_catalog_round_3')
DATASETS = [
    {
        'name': 'human',
        'gpt': ROUND3_ROOT / 'human_data/human_perf_prs_with_gpt_analysis_new_full_catalog.csv',
        'gemini': ROUND3_ROOT / 'human_data/human_perf_prs_with_gemini_analysis_new_full_catalog.csv',
        'manual': Path('./Final_Human_PRs.csv'),
        'output': Path('./Round3_Final_Human_PRs.csv'),
    },
    {
        'name': 'agent',
        'gpt': ROUND3_ROOT / 'agent_data/ai_perf_prs_with_gpt_analysis_new_full_catalog.csv',
        'gemini': ROUND3_ROOT / 'agent_data/ai_perf_prs_with_gemini_analysis_new_full_catalog.csv',
        'manual': Path('./Final_Agent_PRs.csv'),
        'output': Path('./Round3_Final_Agent_PRs.csv'),
    },
]


def normalize(value: str) -> str:
    return value.strip() if value else ''


def load_llm_rows(path: Path) -> dict:
    rows = {}
    with path.open(newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            pr_id = row['id'].strip()
            rows[pr_id] = {
                'url': row.get('html_url', '').strip(),
                'pattern': normalize(row.get('optimization_pattern', '')),
                'subpattern': normalize(row.get('optimization_subpattern', '')),
            }
    return rows


def load_manual_rows(path: Path) -> dict:
    rows = {}
    with path.open(newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            pr_id = row['PR ID'].strip()
            rows[pr_id] = {
                'url': row.get('PR URL', '').strip(),
                'pattern': normalize(row.get('Final_Pattern', '')),
                'subpattern': normalize(row.get('Final_Sub_Pattern', '')),
            }
    return rows


def write_output(path: Path, rows: list):
    path.parent.mkdir(parents=True, exist_ok=True)
    fieldnames = ['PR ID', 'PR URL', 'Final Pattern', 'Final Sub Pattern', 'Label Source']
    with path.open('w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)


final_label_tables = {}
final_label_stats = {}

for cfg in DATASETS:
    gpt_rows = load_llm_rows(cfg['gpt'])
    gemini_rows = load_llm_rows(cfg['gemini'])
    manual_rows = load_manual_rows(cfg['manual'])

    combined = []
    stats = {'llm_agreement': 0, 'manual_override': 0, 'fallback': 0}
    seen = set()

    for pr_id, gpt_row in gpt_rows.items():
        gemini_row = gemini_rows.get(pr_id, {})
        if (
            gpt_row['pattern'] == gemini_row.get('pattern', '')
            and gpt_row['subpattern'] == gemini_row.get('subpattern', '')
        ):
            combined.append({
                'PR ID': pr_id,
                'PR URL': gpt_row['url'] or gemini_row.get('url', ''),
                'Final Pattern': gpt_row['pattern'],
                'Final Sub Pattern': gpt_row['subpattern'],
                'Label Source': 'llm_agreement',
            })
            stats['llm_agreement'] += 1
        elif pr_id in manual_rows:
            manual_row = manual_rows[pr_id]
            combined.append({
                'PR ID': pr_id,
                'PR URL': manual_row['url'] or gpt_row['url'] or gemini_row.get('url', ''),
                'Final Pattern': manual_row['pattern'],
                'Final Sub Pattern': manual_row['subpattern'],
                'Label Source': 'manual_review',
            })
            stats['manual_override'] += 1
        else:
            combined.append({
                'PR ID': pr_id,
                'PR URL': gpt_row['url'] or gemini_row.get('url', ''),
                'Final Pattern': gpt_row['pattern'],
                'Final Sub Pattern': gpt_row['subpattern'],
                'Label Source': 'llm_fallback',
            })
            stats['fallback'] += 1
        seen.add(pr_id)

    for pr_id, manual_row in manual_rows.items():
        if pr_id not in seen:
            combined.append({
                'PR ID': pr_id,
                'PR URL': manual_row['url'],
                'Final Pattern': manual_row['pattern'],
                'Final Sub Pattern': manual_row['subpattern'],
                'Label Source': 'manual_only',
            })
            stats['manual_override'] += 1
            seen.add(pr_id)

    combined.sort(key=lambda row: row['PR ID'])
    write_output(cfg['output'], combined)
    final_label_tables[cfg['name']] = combined
    final_label_stats[cfg['name']] = stats
    print(
        f"{cfg['name'].title()} final labels: {len(combined)} PRs "
        f"(LLM agreement: {stats['llm_agreement']}, manual: {stats['manual_override']}, "
        f"fallback: {stats['fallback']}). Saved to {cfg['output'].name}."
    )

for name, rows in final_label_tables.items():
    print(f"{name.title()} sample rows:")
    for row in rows[:3]:
        print(row)


Human final labels: 83 PRs (LLM agreement: 43, manual: 40, fallback: 0). Saved to Round3_Final_Human_PRs.csv.
Agent final labels: 324 PRs (LLM agreement: 169, manual: 155, fallback: 0). Saved to Round3_Final_Agent_PRs.csv.
Human sample rows:
{'PR ID': '2260441374', 'PR URL': 'https://github.com/OpenHFT/Chronicle-Core/pull/684', 'Final Pattern': 'Control-Flow and Branching Optimizations', 'Final Sub Pattern': 'Rearranging for early return', 'Label Source': 'manual_review'}
{'PR ID': '2260678480', 'PR URL': 'https://github.com/OpenHFT/Chronicle-Wire/pull/984', 'Final Pattern': 'I/O and Synchronization', 'Final Sub Pattern': 'Concurrency Control / Lock Optimization', 'Label Source': 'llm_agreement'}
{'PR ID': '2269202548', 'PR URL': 'https://github.com/seasonedcc/remix-forms/pull/272', 'Final Pattern': 'Code Smells and Structural Simplification', 'Final Sub Pattern': 'Remove Unnecessary Method Calls', 'Label Source': 'manual_review'}
Agent sample rows:
{'PR ID': '2766896431', 'PR URL': 'h